In [ ]:
# --- Cell 0: Load Source Data ---
%matplotlib widget

import opym
from pathlib import Path
import importlib
import sys

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

# Reload modules
try:
    importlib.reload(opym.dataloader)
    importlib.reload(opym.viewer)
    importlib.reload(opym.petakit)
    importlib.reload(opym)
    print("Reloaded opym.dataloader, opym.viewer, and opym.petakit")
except Exception as e:
    print(f"Note: Could not reload all modules: {e}")


# --- Create Interactive Widgets ---
print("\n1. Select your 'processed_tiff_series_split' directory.")
print("2. Click 'Load Data'.")

dir_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    title="<b>Select Source TIFF Directory (e.g., '.../processed_tiff_series_split'):</b>",
    show_only_dirs=True
)
load_button = widgets.Button(
    description="Load Data", button_style="primary"
)
load_output = widgets.Output()

# --- Global variables to hold data and parameters ---
get_stack_source = None
t_max_source = 0
c_max_source = 0
base_name_source = ""  # <-- ADD THIS
# Default physical params (from config.json)
default_xy_size = 0.136
default_z_step = 1.0
default_angle = 21.72


def on_load_button_clicked(b):
    global get_stack_source, t_max_source, c_max_source, base_name_source
    
    with load_output:
        load_output.clear_output()
        if not dir_chooser.value:
            print("❌ ERROR: No directory selected.")
            return
        
        data_dir = Path(dir_chooser.value)
        try:
            print(f"--- Loading source data from: {data_dir.name} ---")
            
            # 1. Load the *source* TIFF series
            # --- MODIFIED: Get base_name from the log file ---
            log_file = next(data_dir.glob("*_processing_log.json"), None)
            if log_file:
                base_name_source = log_file.stem.replace("_processing_log", "")
            else:
                # Fallback: try to get from a file name
                first_file = next(data_dir.glob("*.tif"), None)
                if first_file:
                    base_name_source = first_file.name.split("_T000")[0]
                else:
                    print("❌ ERROR: Could not find _processing_log.json or any .tif file to get base_name.")
                    return
            # --- End Modification ---

            get_stack, t, z, c, y, x = opym.load_tiff_series(data_dir)
            
            # 2. Store the loader function and max indices
            get_stack_source = get_stack
            t_max_source = t
            c_max_source = c
            
            print(f"✅ Data loaded. Base name: '{base_name_source}'")
            print(f"   Shape: T={t+1}, Z={z+1}, C={c+1}, Y={y}, X={x}")
            print("\nYou can now run Cell 1 to tune parameters.")
            
        except FileNotFoundError as e:
            print(f"\n❌ ERROR: Could not load data.")
            print(f"  Details: {e}")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)

load_button.on_click(on_load_button_clicked)
display(widgets.VBox([dir_chooser, load_button, load_output]))

In [ ]:
# --- Cell 1: Interactive Parameter Tuner ---

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import functools

# --- Create Parameter Sliders ---
t_slider = widgets.IntSlider(
    description="Timepoint (T):", min=0, max=t_max_source, step=1, value=0
)
c_slider = widgets.IntSlider(
    description="Channel (C):", min=0, max=c_max_source, step=1, value=0
)
angle_slider = widgets.FloatSlider(
    value=default_angle,
    min=20.0,
    max=25.0,
    step=0.01,
    description='Sheet Angle:',
    readout_format='.2f',
    style={'description_width': 'initial'},
)
z_step_slider = widgets.FloatSlider(
    value=default_z_step,
    min=0.5,
    max=2.0,
    step=0.05,
    description='Z-Step (um):',
    readout_format='.2f',
    style={'description_width': 'initial'},
)
xy_size_slider = widgets.FloatSlider(
    value=default_xy_size,
    min=0.05,
    max=0.2,
    step=0.001,
    description='XY Pixel Size (um):',
    readout_format='.3f',
    style={'description_width': 'initial'},
)

# --- Create Buttons and Output Areas ---
run_view_button = widgets.Button(
    description="Run & View Processed Stack", button_style="success"
)
log_output = widgets.Output(layout={'border': '1px solid black', 'height': '200px', 'overflow_y': 'scroll'})
view_output = widgets.Output()

# --- Define Button Click Handler ---
def on_run_view_button_clicked(b):
    # Disable button
    run_view_button.disabled = True
    
    with log_output:
        clear_output(wait=True)
    with view_output:
        clear_output(wait=True)

    try:
        if get_stack_source is None:
            with log_output:
                print("❌ ERROR: Source data not loaded. Please run Cell 0 first.")
            return

        # 1. Get all parameters from widgets
        t = t_slider.value
        c = c_slider.value
        angle = angle_slider.value
        z_step = z_step_slider.value
        xy_size = xy_size_slider.value
        
        with log_output:
            print(f"Processing T={t}, C={c} with Angle={angle}, Z-Step={z_step}...")

        # 2. Load the single 3D source stack
        source_stack = get_stack_source(t, c)
        if source_stack is None or np.max(source_stack) == 0:
            with log_output:
                print(f"❌ ERROR: Source stack T={t}, C={c} is empty or not found.")
            return

        # 3. Run processing (this now uses the temp directory method)
        processed_stack = opym.petakit.tune_single_stack(
            source_stack,
            base_name=base_name_source, # <-- Pass the base_name
            xy_pixel_size=xy_size,
            z_step_um=z_step,
            sheet_angle_deg=angle,
            interp_method="linear",
        )
        
        with log_output:
            print(f"✅ Processing complete. Final stack shape: {processed_stack.shape}")
            print("Displaying viewer...")
        
        # 4. --- Display the result ---
        # Create a "dummy" get_stack function
        
        @functools.lru_cache(maxsize=1)
        def get_processed_stack_wrapper(t_idx, c_idx):
            return processed_stack

        # Get the new shape for the viewer
        z_max_new, y_new, x_new = processed_stack.shape
        
        with view_output:
            # Call the viewer with our wrapper function
            opym.single_channel_viewer(
                get_stack=get_processed_stack_wrapper,
                T_max=0,  # Only one timepoint
                Z_max=z_max_new - 1,
                C_max=0,  # Only one channel
                Y=y_new,
                X=x_new,
            )

    except Exception as e:
        with log_output:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)
    finally:
        # Re-enable the button
        run_view_button.disabled = False


# --- Link button and display UI ---
run_view_button.on_click(on_run_view_button_clicked)

# --- Update sliders to use global max values ---
t_slider.max = t_max_source
c_slider.max = c_max_source
# ---------------------------------------------

param_box = widgets.VBox([
    widgets.HBox([t_slider, c_slider]),
    angle_slider,
    z_step_slider,
    xy_size_slider
])

display(widgets.VBox([param_box, run_view_button, log_output, view_output]))